Implementarea unui Perceptron Simplu _de Mihai Dan Nadăș (mihai.nadas@ubbcluj.ro), ianuarie 2025_

Acest notebook implementează o versiune a perceptronului așa cum a fost introdusă în lucrarea din 1958 a lui Frank Rosenblatt, "The Perceptron: A Probabilistic Model for Information Storage and Organization in the Brain".

Vom încerca să utilizăm concepte matematice de bază, evitând algebra liniară (adică lucrul cu vectori și matrici) pe cât posibil.

## Obiectiv

Scopul este de a antrena un model cu două greutăți, $w_{1},\ w_{2}$, câte una pentru fiecare dintre coordonatele $x,\ y$ ale unui punct definit ca $(x,\ y)$, și un bias $b$, folosind o adaptare a ecuației algebrice $y = mx + c$ a formei panta-intercepția a unei drepte.

Modelul va rezolva o sarcină simplă de clasificare, a unui set de date separabil liniar pe baza următoarei funcții:

$
f: \mathbb{N} \to \mathbb{N}, \quad f(x) =
\begin{cases}
x & \text{dacă } x \bmod 2 = 0, \\
2x & \text{dacă } x \bmod 2 = 1.
\end{cases}
$

## Set de date
Mai întâi, vom genera un set de date, folosind Biblioteca Standardă Python.

In [ ]:
```python
import random


def generate_dataset(num_items=20, start=0, stop=100):
    random.seed(42)
    dataset = []
    x1_values = set()
    while len(dataset) < num_items:
        x1 = random.randint(start, stop)
        if x1 in x1_values:
            continue
        x1_values.add(x1)
        x2 = x1 if x1 % 2 == 0 else 2 * x1
        y = (
            0 if x1 == x2 else 1
        )  # (x1, x2) este etichetat ca fiind Clasa 0 dacă x1 este par și Clasa 1 în caz contrar
        dataset.append((x1, x2, y))
    return dataset


dataset = generate_dataset()

# să împărțim acum datasetul în seturi de antrenament și test
train_ratio = 0.8
num_train = int(len(dataset) * train_ratio)
dataset_train, dataset_test = dataset[:num_train], dataset[num_train:]
print(f"Set de antrenament (n={len(dataset_train)}): {dataset_train}")
print(f"Set de test (n={len(dataset_test)}: {dataset_test}")
```

## Reprezentare Vizuală

Folosind _Matplotlib_ și _pandas_, vom reprezenta vizual seturile de date de antrenament și testare.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


def plot_datasets(train_dataset, test_dataset):
    # Combină seturile de date într-un DataFrame pentru o manipulare mai ușoară
    train_df = pd.DataFrame(train_dataset, columns=["x1", "x2", "class"])
    train_df["set"] = "Train"

    test_df = pd.DataFrame(test_dataset, columns=["x1", "x2", "class"])
    test_df["set"] = "Test"

    combined_df = pd.concat([train_df, test_df], ignore_index=True)

    # Definește culorile și marcajele
    colors = {0: "blue", 1: "red"}
    markers = {"Train": "o", "Test": "x"}

    # Plotează fiecare grup folosind Matplotlib
    fig, ax = plt.subplots()
    for (dataset, cls), group in combined_df.groupby(["set", "class"]):
        ax.scatter(
            group["x1"],
            group["x2"],
            color=colors[cls],
            label=f"{dataset} Dataset, Class {cls}",
            s=30,
            marker=markers[dataset],
        )

    # Gestionează legenda și etichetele
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), title="Dataset și Clasă", loc="best")
    ax.set_xlabel("x1")
    ax.set_ylabel("x2")
    ax.set_title("Seturi de date pentru antrenament și testare")
    ax.grid(True)


plot_datasets(dataset_train, dataset_test)
plt.show()

## Definirea unui Clasificator Liniar

Cu dataset-ul nostru pregătit, ne îndreptăm acum atenția către fundația matematică care permite modelului nostru să clasifice o intrare $x_{1} \text{și} x_{2}$ ca aparținând claselor $0$ sau $1$, după cum urmează:

$
c: \mathbb{N} \to \{0,1\}, \quad
c(x_{1},x_{2}) =
\begin{cases} 
1, & \text{dacă } (x_{1},x_{2}) \in \text{Clasa 1}, \\
0, & \text{dacă } (x_{1},x_{2}) \in \text{Clasa 2}.
\end{cases}
$

Această clasificare poate fi realizată utilizând reprezentarea algebrică a unei linii într-un sistem de coordonate cartezian, descrisă de:

$
z(x) = w_{1}x_{1}+w_{2}x_{2} + c,
$

unde:
- $w_{1} \text{ și } w_{2}$ reprezintă ponderile care determină panta ce reprezintă unghiul liniei rezultate față de axa $x$,
- $c$ este termenul liber, indicând unde linia intersectează axa $y$.

Din graficul reprezentat mai sus, devine clar că cele două clase sunt liniar separabile, ceea ce face adecvată utilizarea unei frontiere de separare liniară unde $w_{1} \text{ și } w_{2}$ sunt antrenate folosind algoritmul Perceptron al lui Rosenblatt.

Pentru a ilustra acest punct, iată cum ar arăta o linie definită de $w_{1}=1, w_{2}=0.5, \text{ și } c=0$ pe imaginea noastră anterioară.

In [ ]:
```python
zx = lambda x1, x2, w1, w2, c: w1 * x1 + w2 * x2 + c


def plot_zx(w1, w2, c):
    x2 = lambda x1: (
        (-w1 * x1 - c) / w2 if w2 != 0 else -c / w1 if w1 != 0 else c
    )  # acest lucru se datorează faptului că ecuația dreptei este w1*x1 + w2*x2 + c = 0, deci x2 = (-w1*x1 - c) / w2
    x1_values = range(0, 101)
    x2_values = [x2(x1) for x1 in x1_values]
    plt.plot(x1_values, x2_values, label=f"{w1}x1+{w2}x2+{c}=0")
    plt.legend(loc="best")


def plot_datasets_and_zx(w1, w2, c):
    plot_datasets(dataset_train, dataset_test)
    plot_zx(w1, w2, c)


plot_datasets_and_zx(-1.5, 1.1, -10)
```

Acum, este evident că, în această configurație, punctele de date sunt separate clar. Totuși, există și modalități alternative de a configura $w_{1},\ w_{2}, \text{ și } c$ pentru a obține exemple mai puțin ideale. De exemplu, când $w_{1}=0.1,\ w_{2}=0.1, \text{ și } c=0.5$, obținem o limită de separare mai puțin ideală.

In [ ]:
plot_datasets_and_zx(0.1, 0.1, 0.5)

În acest caz particular, $z$ nu va ajuta la clasificarea niciunui punct de date.

## Evaluarea performanței clasificatorului

Acum că am definit $z=w_{1}x_{1}+w_{2}x_{2}+c$ ca fiind limita decizională a clasificatorului nostru, și că am stabilit vizual că funcționează pentru unele valori alese (dintre, probabil, alte opțiuni), să definim o abordare computațională pentru a determina performanța acestuia, folosind o metrică numită _acuratețe_.

### Definirea clasificatorului

Dar înainte de a aprofunda performanța clasificatorului nostru, să-l definim astfel:

$
c: \mathbb{N} \to \{0,1\}, \quad
c(x_{1},x_{2}) =
\begin{cases} 
1, & \text{dacă } z(x_{1},x_{2}) >= 0, \\
0, & \text{dacă } z(x_{1},x_{2}) <0.
\end{cases}
$

În esență, aceasta înseamnă că dacă un număr $x$ este deasupra limitei decizionale definite de $f(x)$, va fi clasificat ca $1$, altfel ca $0$.

Să implementăm clasificatorul în cod și apoi să revenim la discuția despre evaluarea performanței sale.

In [ ]:
cx = lambda x1, x2, w1, w2, c: 1 if zx(x1, x2, w1, w2, c) >= 0 else 0


def accuracy(dataset, w1, w2, c):
    print(f"Se calculează acuratețea pe setul de antrenament folosind w1={w1}, w2={w2}, c={c}")
    correct = 0
    for x1, x2, y in dataset:
        if y == cx(x1, x2, w1, w2, c):
            correct += 1
    print(
        f"Acuratețea rezultată: {correct}/{len(dataset)}, adică {correct/len(dataset)*100:.2f}%"
    )
    return correct / len(dataset)


# Aplicarea funcției de acuratețe pe setul de antrenament folosind cele două seturi de greutăți și bias așa cum s-a arătat mai sus, în primul exemplu
accuracy(dataset_train, -1.5, 1.1, -10)

# Aplicarea funcției de acuratețe pe setul de antrenament folosind două seturi de greutăți și bias așa cum s-a arătat mai sus, în al doilea exemplu
accuracy(dataset_train, 0.1, 0.1, 0.5)

### Discuție despre Acuratețe

Așa cum s-a arătat anterior, modificarea valorilor pentru greutăți și bias generează rezultate de acuratețe diferite. Acest lucru se datorează faptului că greutățile și biasurile distincte stabilesc limite de decizie unice care pot funcționa diferit, în funcție de cât de bine "clasifică" dataset-ul în clasele corecte. Provocarea constă în găsirea valorilor "optime" pentru acești doi parametri. Pentru a realiza acest lucru, folosim un proces cunoscut sub numele de _antrenare a modelului_.

## Antrenarea Modelului

Utilizând concluziile obținute din analiza noastră, vom continua acum să ne antrenăm modelul printr-o serie de iterații, rafinând valorile greutăților și biasului până când ajungem la un nivel acceptabil de acuratețe. Acest proces iterativ ne permite să găsim combinația ideală de parametri care să se potrivească cel mai bine dataset-ului nostru și sarcinii de clasificare.

In [ ]:
# Mai întâi, să inițializăm ponderile și bias-ul la zero
w1, w2, c = 0, 0, 0

# Acum să definim rata de învățare
learning_rate = 0.1

# Acum să definim numărul de epoci
num_epochs = 100

# Creăm un DataFrame pentru a stoca detaliile epocilor
epoch_details = pd.DataFrame(columns=["epoch", "x1", "x2", "y", "z", "y_hat", "w1", "w2", "c"])

# Acum să începem bucla de antrenament
for epoch in range(num_epochs):
    print(f"Epoca {epoch+1}")
    for x1, x2, y in dataset_train:
        z = zx(x1, x2, w1, w2, c)
        y_hat = 1 if z >= 0 else 0
        w1 += learning_rate * (y - y_hat) * x1
        w2 += learning_rate * (y - y_hat) * x2
        c += learning_rate * (y - y_hat)
        print(f"  x1={x1}, x2={x2}, y={y}, z={z:.2f}, y_hat={y_hat}, w1={w1:.2f}, w2={w2:.2f}, c={c:.2f}")
        # Adăugăm detaliile în DataFrame
        epoch_details = epoch_details.concat({
            "epoch": epoch + 1,
            "x1": x1,
            "x2": x2,
            "y": y,
            "z": z,
            "y_hat": y_hat,
            "w1": w1,
            "w2": w2,
            "c": c
        }, ignore_index=True)

# Afișăm DataFrame-ul
epoch_details.head()